### Week 5 Final Report

#### Opening a New Restaurant in Colombo, Sri Lanka

#### Importing the necessary libraries.

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!pip install geocoder
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

print("Libraries imported.")

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: / 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
                                                                    \                                                     failed

UnsatisfiableError: The following specifications were found
to be incompatible with the existing python installation in your environment:

Specifications:

  - cffi -> python[version='2.7.*|3.5.*|3.6.*|3.6.12|3.6.12|>=3.6,<3.7.0a0|>=3.7,<3.8.0a0|>=3.9,<3.10.0a0|>=3.8,<3.9.0a0|3.7.9|3.6.9|3.6.9|3.6.9|>=2.7,<2.8.0a0|3.6.9|>=3.5,<3.6.0a0|3.4.*',build='1_73_pypy|5_73_pypy|4_73_pypy|3_73_pypy|2_73_pypy|0_73_pypy']
  - geopy -> python[version='2.7.*|3.4.*|3.5.*|3

#### Scrap the data from Wikipedia page into a DataFrame

In [3]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_of_Colombo").text

In [4]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [5]:
# create a list to store neighborhood data
neighborhoodList = []

In [6]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [7]:
# create a new DataFrame from the list
cl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

cl_df.head()

,Neighborhood
0,Areas of Colombo
1,Athurugiriya
2,Bambalapitiya
3,Battaramulla
4,Batuwatta


In [8]:
# print the number of rows of the dataframe
cl_df.shape

(67, 1)

#### To get coordinates

In [80]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Colombo, Sri Lanka'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [12]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in cl_df["Neighborhood"].tolist() ]

In [13]:
coords

[[3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.1538600000000656, 101.70666000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.69405000000006],
 [3.1430900000000292, 101.68653000000006],
 [3.147890000000075, 101.69405000000006],
 [3.147890000000075, 101.6940500

In [14]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [15]:
# merge the coordinates into the original dataframe
cl_df['Latitude'] = df_coords['Latitude']
cl_df['Longitude'] = df_coords['Longitude']

In [16]:
print(cl_df.shape)
cl_df

(67, 3)


,Neighborhood,Latitude,Longitude
0,Areas of Colombo,3.147890,101.694050
1,Athurugiriya,3.147890,101.694050
2,Bambalapitiya,3.147890,101.694050
3,Battaramulla,3.147890,101.694050
4,Batuwatta,3.147890,101.694050
5,Bloemendhal,3.147890,101.694050
6,Boralesgamuwa,3.147890,101.694050
7,Borella,3.147890,101.694050
8,Cinnamon Gardens,3.153860,101.706660
9,Colombo,3.147890,101.694050


In [17]:
# save the DataFrame as CSV file
cl_df.to_csv("cl_df.csv", index=False)

 Create a map of Colombo with neighborhoods superimposed on top

In [59]:
address = 'Colombo, Sri Lanka'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

6.9349969 79.8538463


In [60]:
# create map of Colombo using latitude and longitude values
map_cl = folium.Map(location=[latitude, longitude], zoom_start=11)


# add markers to map
for lat, lng, neighborhood in zip(cl_df['Latitude'], cl_df['Longitude'], cl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_cl)  
    
map_cl

In [61]:
# save the map as HTML file
map_cl.save('map_cl.html')

#### Use the Foursquare API to explore the neighborhoods

In [62]:
# define Foursquare Credentials and Version
CLIENT_ID = 'QCKZNZTKOBJL1XZQJLBHHFBEI35S01TY0P03DYXXJ31QPQNK' # your Foursquare ID
CLIENT_SECRET = '4WRW3XZQX1EO4G0NKDFXO2IKQ0IYVS1FH2CMMHADS025MUBF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QCKZNZTKOBJL1XZQJLBHHFBEI35S01TY0P03DYXXJ31QPQNK
CLIENT_SECRET:4WRW3XZQX1EO4G0NKDFXO2IKQ0IYVS1FH2CMMHADS025MUBF


In [63]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(cl_df['Latitude'], cl_df['Longitude'], cl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [64]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(6700, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Areas of Colombo,3.14789,101.69405,Urbanscapes House,3.146803,101.696028,Exhibit
1,Areas of Colombo,3.14789,101.69405,BackHome Kuala Lumpur,3.148732,101.697887,Hostel
2,Areas of Colombo,3.14789,101.69405,Shuang Xi 雙喜,3.142853,101.696465,Speakeasy
3,Areas of Colombo,3.14789,101.69405,Bangunan Sultan Abdul Samad,3.148620,101.694549,Monument / Landmark
4,Areas of Colombo,3.14789,101.69405,Shin Kee Beef Noodles,3.145058,101.696877,Noodle House


#### Analyzing Each Neighborhood

In [65]:
cl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
cl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [cl_onehot.columns[-1]] + list(cl_onehot.columns[:-1])
cl_onehot = cl_onehot[fixed_columns]

print(cl_onehot.shape)
cl_onehot.head()

(6700, 158)


,Neighborhoods,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Aquarium,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bakery,Bar,Basketball Court,Beer Bar,Beer Garden,Bistro,Bookstore,Boutique,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Café,Camera Store,Cantonese Restaurant,Chettinad Restaurant,Chinese Breakfast Place,Chinese Restaurant,Circus,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Stadium,Concert Hall,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Escape Room,Exhibit,Fabric Shop,Fast Food Restaurant,Fishing Store,Flea Market,Flower Shop,Food,Food Court,Food Service,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Garden,General Entertainment,Gift Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site,Hockey Arena,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Iraqi Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Leather Goods Store,Lounge,Malay Restaurant,Mamak Restaurant,Massage Studio,Mediterranean Restaurant,Men's Store,Middle Eastern Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Night Market,Noodle House,Office,Other Great Outdoors,Outlet Store,Pakistani Restaurant,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Poke Place,Pool,Pool Hall,Pub,Residential Building (Apartment / Condo),Resort,Restaurant,Sake Bar,Salon / Barbershop,Sandwich Place,Satay Restaurant,Science Museum,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Snack Place,Soup Place,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Stadium,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Track Stadium,Trail,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Women's Store,Yakitori Restaurant
0,Areas of Colombo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Areas of Colombo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Areas of Colombo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Areas of Colombo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Areas of Colombo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [66]:
cl_grouped = cl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(cl_grouped.shape)
cl_grouped

(67, 158)


,Neighborhoods,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Aquarium,Arcade,Art Gallery,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Bakery,Bar,Basketball Court,Beer Bar,Beer Garden,Bistro,Bookstore,Boutique,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Café,Camera Store,Cantonese Restaurant,Chettinad Restaurant,Chinese Breakfast Place,Chinese Restaurant,Circus,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Stadium,Concert Hall,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Escape Room,Exhibit,Fabric Shop,Fast Food Restaurant,Fishing Store,Flea Market,Flower Shop,Food,Food Court,Food Service,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Garden,General Entertainment,Gift Shop,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Historic Site,Hockey Arena,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Iraqi Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Leather Goods Store,Lounge,Malay Restaurant,Mamak Restaurant,Massage Studio,Mediterranean Restaurant,Men's Store,Middle Eastern Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Night Market,Noodle House,Office,Other Great Outdoors,Outlet Store,Pakistani Restaurant,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Poke Place,Pool,Pool Hall,Pub,Residential Building (Apartment / Condo),Resort,Restaurant,Sake Bar,Salon / Barbershop,Sandwich Place,Satay Restaurant,Science Museum,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Snack Place,Soup Place,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Stadium,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Track Stadium,Trail,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Women's Store,Yakitori Restaurant
0,Areas of Colombo,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.01,0.02,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.01,0.00,0.00,0.00,0.00,0.07,0.00,0.00,0.01,0.00,0.02,0.01,0.01,0.01,0.01,0.03,0.00,0.00,0.01,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.01,0.01,0.02,0.00,0.00,0.00,0.00,0.00,0.02,0.09,0.03,0.01,0.00,0.06,0.00,0.00,0.02,0.00,0.00,0.01,0.01,0.00,0.02,0.00,0.00,0.05,0.01,0.00,0.00,0.01,0.01,0.03,0.01,0.00,0.00,0.02,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.04,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.01,0.00,0.00,0.01,0.02,0.01,0.01,0.01,0.01,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.01,0.00,0.01,0.00,0.01,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00
1,Athurugiriya,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.01,0.02,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.01,0.00,0.00,0.00,0.00,0.07,0.00,0.00,0.01,0.00,0.02,0.01,0.01,0.01,0.01,0.03,0.00,0.00,0.01,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.01,0.01,0.02,0.00,0.00,0.00,0.00,0.00,0.02,0.09,0.03,0.01,0.00,0.06,0.00,0.00,0.02,0.00,0.00,0.01,0.01,0.00,0.02,0.00,0.00,0.05,0.01,0.00,0.00,0.01,0.01,0.03,0.01,0.00,0.00,0.02,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.04,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.03,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.01,0.00,0.00,0.01,0.02,0.01,0.01,0.01,0.01,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.01,0.00,0.01,0.00,0.01,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00
2,Bambalapitiya,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.01,0.02,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.01,0.00,0.00,0.00,0.00,0.07,0.00,0.00,0.01,0.00,0.02,0.0

In [67]:
len(cl_grouped[cl_grouped["Restaurant"] > 0])

67

#### New DataFrame for only Restaurant data

In [68]:
cl_mall = cl_grouped[["Neighborhoods","Restaurant"]]
cl_mall.head()

,Neighborhoods,Restaurant
0,Areas of Colombo,0.03
1,Athurugiriya,0.03
2,Bambalapitiya,0.03
3,Battaramulla,0.03
4,Batuwatta,0.03


#### Using Kmeans Machine Learning Algorithm for the clustering of the Restaurants

In [69]:
# set number of clusters
kclusters = 3

cl_clustering = cl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(cl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 2, 0], dtype=int32)

In [70]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
cl_merged = cl_mall.copy()

# add clustering labels
cl_merged["Cluster Labels"] = kmeans.labels_


In [71]:
cl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
cl_merged.head()

,Neighborhood,Restaurant,Cluster Labels
0,Areas of Colombo,0.03,0
1,Athurugiriya,0.03,0
2,Bambalapitiya,0.03,0
3,Battaramulla,0.03,0
4,Batuwatta,0.03,0


In [72]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
cl_merged = cl_merged.join(cl_df.set_index("Neighborhood"), on="Neighborhood")

print(cl_merged.shape)
cl_merged.head() # check the last columns!

(67, 5)


,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
0,Areas of Colombo,0.03,0,3.14789,101.69405
1,Athurugiriya,0.03,0,3.14789,101.69405
2,Bambalapitiya,0.03,0,3.14789,101.69405
3,Battaramulla,0.03,0,3.14789,101.69405
4,Batuwatta,0.03,0,3.14789,101.69405


In [73]:
# sort the results by Cluster Labels
print(cl_merged.shape)
cl_merged.sort_values(["Cluster Labels"], inplace=True)
cl_merged

(67, 5)


,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
0,Areas of Colombo,0.03,0,3.147890,101.694050
34,Kotikawatta,0.03,0,3.147890,101.694050
35,Kottawa,0.03,0,3.147890,101.694050
36,Madampitiya,0.03,0,3.147890,101.694050
38,Maharagama,0.03,0,3.147890,101.694050
39,Malabe,0.03,0,3.147890,101.694050
40,Maradana,0.03,0,3.147890,101.694050
41,Mattakkuliya,0.03,0,3.147890,101.694050
42,Modara,0.03,0,3.147890,101.694050
43,Moratuwa,0.03,0,3.147890,101.694050


In [74]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(cl_merged['Latitude'], cl_merged['Longitude'], cl_merged['Neighborhood'], cl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [75]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

#### Examining Clusters

#### Cluster 0

In [76]:
cl_merged.loc[cl_merged['Cluster Labels'] == 0]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
0,Areas of Colombo,0.03,0,3.147890,101.694050
34,Kotikawatta,0.03,0,3.147890,101.694050
35,Kottawa,0.03,0,3.147890,101.694050
36,Madampitiya,0.03,0,3.147890,101.694050
38,Maharagama,0.03,0,3.147890,101.694050
39,Malabe,0.03,0,3.147890,101.694050
40,Maradana,0.03,0,3.147890,101.694050
41,Mattakkuliya,0.03,0,3.147890,101.694050
42,Modara,0.03,0,3.147890,101.694050
43,Moratuwa,0.03,0,3.147890,101.694050


#### Cluster 1

In [78]:
cl_merged.loc[cl_merged['Cluster Labels'] == 1]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
33,Kotahena,0.05,1,3.18517,101.67643
37,Maha Nuge Gardens,0.05,1,3.21536,101.64020


#### Cluster 2

In [79]:
cl_merged.loc[cl_merged['Cluster Labels'] == 2]

,Neighborhood,Restaurant,Cluster Labels,Latitude,Longitude
53,"Pettah, Sri Lanka",0.01,2,3.07927,101.735127
8,Cinnamon Gardens,0.01,2,3.15386,101.706660


#### The observations that can be made are:

Cluster 1 has a greater number of restaurants present compared to the other clusters, so starting up a restaurant there might not be a good idea has there'll be a high level of competition, thereby leading to less sales piotential.
Whereas, Cluster 0 presents a moderate number of restaurants, opening a restaurant here might be feasible as the competition wouldn't be so fierce but as a newly opening restaurant, it might not be the best idea as customers might not quickly start coming around.
On the other hand, Cluster 2 has very little restaurants available, these are good locations to start up a restaurant as the market is very scarce. Therefore, there is a high potential of making a lot of sales in these areas.
Based on this project, it's recommended that Cluster 2 be considered as the most viable location to start up a new restaurant.